In [1]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

chat_model = ChatNVIDIA(
  model="meta/llama-3.1-8b-instruct",
  api_key=os.environ["NVIDIA_API_KEY"], 
  temperature=0.5,
)

In [3]:
from typing import Sequence, Literal
from typing_extensions import Annotated, TypedDict

from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, RemoveMessage
from langgraph.graph.message import add_messages

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph, END

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    summary: str

def call_model(state: State):
    summary =  state.get("summary", "")
    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else: 
        messages = state["messages"]

    response = chat_model.invoke(messages)

    return {"messages": [response]}

def should_summarize(state: State) -> Literal["summarize_conversation", END]: # type: ignore
    messages = state["messages"]
    if len(messages) > 6:
        return "summarize_conversation"
    return END

def summarize_conversation(state: State):
    summary =  state.get("summary", "")
    if summary:
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = chat_model.invoke(messages)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]

    return {"summary": response.content, "messages": delete_messages}

# Define a new graph
workflow = StateGraph(State)

# Define the conversation node and the summarize node
workflow.add_edge(START, "conversation")
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)
workflow.add_conditional_edges("conversation", should_summarize)
workflow.add_edge("summarize_conversation", END)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [4]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

def visualize_graph():
    return display(
        Image(
            app.get_graph().draw_mermaid_png(
                draw_method=MermaidDrawMethod.API,
            )
        )
    )

In [5]:
def print_update(update):
    for key, value in update.items():
        for message in value["messages"]:
            message.pretty_print()
        if "summary" in value:
            print(value["summary"])

In [6]:
import getpass

thread_id = getpass.getpass()
config = {"configurable": {"thread_id": thread_id}}
input_message = HumanMessage(content="Hi! I'm bob")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="What's my name?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

input_message = HumanMessage(content="Tell me about your-self")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

Hi! I'm bob
================================== Ai Message ==================================

Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?
================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob!
================================ Human Message =================================

Tell me about your-self
================================== Ai Message ==================================

I'm an artificial intelligence model, which means I'm a computer program designed to simulate conversation and answer questions to the best of my ability. I don't have a physical body or a personal life like a human would, but I'm here to help you with any questions or topics you'd like to discuss.

I was created to assist and communicate 

In [7]:
values = app.get_state(config).values
values

{'messages': [HumanMessage(content="Hi! I'm bob", additional_kwargs={}, response_metadata={}, id='df80f69f-e239-4ad4-a0d2-259b7e01ee77'),
  AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", 'token_usage': {'prompt_tokens': 16, 'total_tokens': 41, 'completion_tokens': 25}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-8b-instruct'}, id='run-1269e014-614b-4a80-81e4-ee846a03c3d7-0', usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41}, role='assistant'),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='df2ae448-66e9-4137-9ad2-e11e7bab02a5'),
  AIMessage(content='Your name is Bob!', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'Your name is Bob!', 'toke

In [8]:
input_message = HumanMessage(content="What do you like?")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

What do you like?
================================== Ai Message ==================================

I don't have personal preferences or feelings like humans do. I'm designed to be neutral and provide information on a wide range of topics. However, I can tell you about things that people often enjoy or find interesting!

Many people enjoy learning about:

* Space and astronomy
* Science and technology
* History and culture
* Music and art
* Travel and exploration
* Food and cooking

I can provide information and answer questions on these topics and many more. What are your interests, Bob?
================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message ================================


================================ Remove Message =========================

In [9]:
values = app.get_state(config).values
print(values['summary'])

Here's a summary of our conversation:

* We introduced ourselves, and I learned that your name is Bob.
* You asked me to tell you about myself, and I explained that I'm an artificial intelligence model designed to provide information and assist with conversations.
* You asked me what I like, and I explained that I don't have personal preferences or feelings, but I can provide information on a wide range of topics that people often enjoy.
* Finally, you asked me to summarize our conversation, which I've just done!


In [10]:
input_message = HumanMessage(content="What is my name")
input_message.pretty_print()
for event in app.stream({"messages": [input_message]}, config, stream_mode="updates"):
    print_update(event)

================================ Human Message =================================

What is my name
================================== Ai Message ==================================

You told me earlier that your name is Bob. Am I correct?
